In [2]:
# let's install the required libraries

!pip install xlrd
!pip install openpyxl
!pip install folium

     |████████████████████████████████| 96 kB 3.1 MB/s eta 0:00:011
     |████████████████████████████████| 243 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 2.6 MB/s eta 0:00:011


In [3]:
# let's import the required libraries

import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [5]:
# let's read the data and create pandas dataframes

data_anagrafica = pd.read_excel('anagrafica_test_250rows.xlsx', engine='openpyxl')
data_annuario = pd.read_excel('annuario_test_250rows.xlsx', engine='openpyxl')
data_miodottore = pd.read_excel('miodottore_test_250rows.xlsx', engine='openpyxl')
data_mybusiness = pd.read_excel('mybusiness_test_250rows.xlsx', engine='openpyxl')

In [6]:
# let's drop unnecessary columns

data_miodottore = data_miodottore.drop(['index','db_miodottore'], axis=1)
data_mybusiness = data_mybusiness.drop(['index','sponsored','db_mybusiness'], axis=1)
data_annuario = data_annuario.drop(['index','localita_annuario','db_annuario'], axis=1)
data_anagrafica = data_anagrafica.drop(['index','db_anagrafica','localita_anagrafica'], axis=1)

In [7]:
# let's do some data cleansing to make our data ready to be analyzed

L = [data_anagrafica, data_annuario, data_miodottore, data_mybusiness]

for j in L:
    for i in j.columns:
        if type(i[0]) == str:
            j[i] = j[i].astype(str).str.lower()

def r(a):
    H = []
    for i in a:
        H.append(i.split())
    return H
ANA = r(data_anagrafica.nome_anagrafica)
ANN = r(data_annuario.nome_annuario)
DOT = r(data_miodottore.nome_miodottore)
BLINK = data_mybusiness.link
BNOME = data_mybusiness.nome_mybusiness

In [8]:
# let's store the observations as keys in a python dictionary

H,K = dict(),[]
for i in range(250):
    if ANA[i][0] not in H:
        H[ANA[i][0]] = [('ANA',i)]
        #K.append([ANA[i][0],('ANA',i)])
    else:
        H[ANA[i][0]].append(('ANA',i))
        #K[]
        
for i in range(250):
    if ANN[i][0] not in H:
        H[ANA[i][0]] = [('ANN',i)]
        #K.append([ANN[i][0],('ANN',i)])
    else:
        H[ANN[i][0]].append(('ANN',i))
        
for i in range(250):
    if DOT[i][0] not in H:
        H[DOT[i][-1]] = [('DOT',i)]
        #K.append([ANN[i][0],('ANN',i)])
    else:
        H[DOT[i][0]].append(('DOT',i))


R = []
for key, value in H.items():
    temp = [key,value]
    R.append(temp)


G = dict()
for i in range(250):
    t = 0
    for j in re.split(r"[, .]", BNOME[i]):
        if j in H:
            H[j].append(('BUS',i))
            t += 1
    if t == 0:
        if BNOME[i] not in G:
            G[BNOME[i]] = [('BUS',i)]
        else:
            G[BNOME[i]].append(('BUS',i))

S = []
for key, value in G.items():
    temp = [key,value]
    S.append(temp)

Q = R+S

In [9]:
# let's rename a few columns
# this step comes handy when we'll concatenate the observations from the 4 datasets

data_anagrafica = data_anagrafica.rename(columns={'cap_anagrafica':'cap','telefono_anagrafica':'telefono','nome_anagrafica':'nome','indirizzo_anagrafica':'indirizzo'})
data_annuario = data_annuario.rename(columns={'nome_annuario':'nome','indirizzo_annuario':'indirizzo','cap_annuario':'cap','telefono_annuario':'telefono'})
data_miodottore = data_miodottore.rename(columns={'nome_miodottore':'nome','indirizzo_miodottore':'indirizzo','cap_miodottore':'cap'})
data_mybusiness = data_mybusiness.rename(columns={'nome_mybusiness':'nome','indirizzo_mybusiness':'indirizzo','cap_mybusiness':'cap'})

In [10]:
# let's insert each observation into one of the following dataframes:
# observations which appear only once into unique_DF, later saved as file A
# observations which seem to appear more than once into multiple_DF, later saved as file B

unique_DF = pd.DataFrame(columns = pd.concat([data_anagrafica,data_annuario,data_miodottore,data_mybusiness]).columns)
multiple_DF = pd.DataFrame(columns = pd.concat([data_anagrafica,data_annuario,data_miodottore,data_mybusiness]).columns)


for i in range(len(Q)):
    if len(Q[i][1]) == 1:
        if Q[i][1][0][0] == 'ANA':
            unique_DF = unique_DF.append(data_anagrafica.loc[[Q[i][1][0][1]]])
        elif Q[i][1][0][0] == 'ANN':
            unique_DF = unique_DF.append(data_annuario.loc[[Q[i][1][0][1]]])
        elif Q[i][1][0][0] == 'DOT':
            unique_DF = unique_DF.append(data_miodottore.loc[[Q[i][1][0][1]]])
        elif Q[i][1][0][0] == 'BUS':
            unique_DF = unique_DF.append(data_mybusiness.loc[[Q[i][1][0][1]]])
    else:
        for j in Q[i][1]:
            #print(j)
            if j[0] == 'ANA':
                multiple_DF = multiple_DF.append(data_anagrafica.loc[[j[1]]])
            elif j[0] == 'ANN':
                multiple_DF = multiple_DF.append(data_annuario.loc[[j[1]]])
            elif j[0] == 'DOT':
                multiple_DF = multiple_DF.append(data_miodottore.loc[[j[1]]])
            elif j[0] == 'BUS':
                multiple_DF = multiple_DF.append(data_mybusiness.loc[[j[1]]])

unique_DF.to_csv('A.csv')
multiple_DF.to_csv('B.csv')

In [11]:
# let's show what unique_DF looks like:

unique_DF

,nome,indirizzo,indirizzo_ori,cap,telefono,latitude,longitude,id_medico,review_miodottore,score_miodottore,...,category,menu,snippet,opening_hours_per_day_monday,opening_hours_per_day_tuesday,opening_hours_per_day_wednesday,opening_hours_per_day_thursday,opening_hours_per_day_friday,opening_hours_per_day_saturday,opening_hours_per_day_sunday
0,perugini marco,"corso regina maria pia, 79, 00100 lido di osti...","corso regina maria pia, 79",122,06.5601495,41.7290663,12.286223,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gereschi andrea,"via di macchia saponara, 66, 00125 roma rm, italy","via di macchia saponara, 66",125,06.5210717,41.7757516,12.3522185,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c.d.m. dentalmedica srl,"via enopide di chio, 15, 00125 roma rm, italy","via enopide di chio, 15",125,06.52169753,41.7618053,12.353569,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,dr. paolo bournens,"via moneti giuseppe 34, roma",NaN,166,NaN,41.8787804,12.3572236,609bab247e2bdf2cccf9320e,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,miniati stefano,"via anassarco, 4, 00124 roma rm, italy","via anassarco, 4",124,06.50914878,41.7572169,12.3566935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,studidentistico dr lombardo nicola,"via di s. costanza, 35",NaN,198,06 4542 4127,41.9223429,12.5153515,NaN,NaN,NaN,...,dentista a roma,nan,nan,chiuso,09–19,10–19,chiuso,10–19,chiuso,chiuso
240,lai putzu odontoiatria & benessere,"via alcide de gasperi, 7",NaN,7100,079 267 0052,40.7203126,8.5800229,NaN,NaN,NaN,...,dentista a sassari,none,none,none,none,none,none,none,none,none
242,cipriano enrico,"via giovanni pascoli, 30",NaN,11,0774 355181,41.9550937,12.716054,NaN,NaN,NaN,...,dentista a tivoli terme,none,none,none,none,none,none,none,none,none
244,dentitalia - dott.ssa zapata dora esther,"via dei fauni, 46",NaN,11,0774 357388,41.9526824,12.7278127,NaN,NaN,NaN,...,dentista a tivoli terme,nan,nan,12–19:30,chiuso,12–19:30,10–19:30,chiuso,chiuso,chiuso


In [16]:
# let's show what multiple_DF looks like:

multiple_DF

,nome,indirizzo,indirizzo_ori,cap,telefono,latitude,longitude,id_medico,review_miodottore,score_miodottore,...,category,menu,snippet,opening_hours_per_day_monday,opening_hours_per_day_tuesday,opening_hours_per_day_wednesday,opening_hours_per_day_thursday,opening_hours_per_day_friday,opening_hours_per_day_saturday,opening_hours_per_day_sunday
212,dr. stefano rossi,"via rizzo luigi 36, roma",NaN,136,NaN,41.9083138,12.4452048,609baa4d7e2bdf2cccf92fb5,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,rossi dr. mario,largo g. la loggia,NaN,149,06 5526 2755,41.8560386,12.446035,NaN,NaN,NaN,...,dentista a roma,nan,nan,09–20,09–20,09–13,09–20,09–20,chiuso,chiuso
202,resta & rossi iommetti dental clinic,"via montello, 30",NaN,195,06 3975 1197,41.9185047,12.4632283,NaN,NaN,NaN,...,dentista a roma,nan,nan,aperto 24 ore su 24,aperto 24 ore su 24,aperto 24 ore su 24,aperto 24 ore su 24,aperto 24 ore su 24,aperto 24 ore su 24,aperto 24 ore su 24
222,amoruso cataldo,"via casali del drago, 29, 00155 roma rm, italy","via dei casali del drago, 29",155,06.22754084,41.8949303,12.5846474,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,centro dentistico solosorrisi parioli,"88, via arno, roma",NaN,198,NaN,41.919468,12.500012,609ba60d7e2bdf2cccf92385,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,dentalpro,via nazionale tiburtina,NaN,12,0774 553460,41.9555079,12.6967062,NaN,NaN,NaN,...,clinica dentale in italia,nan,nan,10–19,10–19,10–19,10–19,10–19,chiuso,chiuso
16,dentalpro,"centro commerciale valecenter, via e. mattei 1...",NaN,30020,041 868 5126,45.5450845,12.3042016,NaN,NaN,NaN,...,clinica dentale a marcon-gaggio zona commerciale,none,none,none,none,none,none,none,none,none
65,dentalpro,"viale dell'oceano pacifico, 83",NaN,144,06 9762 5542,41.8174081,12.4594637,NaN,NaN,NaN,...,clinica dentale a roma,none,none,none,none,none,none,none,none,none
73,dentalpro,"o centro commerciale casetta mattei, via dei s...",NaN,148,06 9762 6760,41.8524743,12.4170058,NaN,NaN,NaN,...,clinica dentale a roma,nan,nan,chiuso,"09:30–13, 15:30–19:30",09:30–19:30,09:30–13,chiuso,"09:30–13, 15:30–19:30","09:30–13, 15:30–19:30"


In [27]:
# let's show the map location of the datapoints across all 4 datasets

map_ = folium.Map()
for i in range(unique_DF.shape[0]):
    folium.Marker([list(unique_DF.latitude)[i], list(unique_DF.longitude)[i]]).add_to(map_)
for i in range(multiple_DF.shape[0]):
    folium.Marker([list(multiple_DF.latitude)[i], list(multiple_DF.longitude)[i]]).add_to(map_)
map_

In [26]:
# let's finally show the map location of the data points from my_business dataframe

bdmap_ = folium.Map()
for i in range(data_mybusiness.shape[0]):
    folium.Marker([data_mybusiness.latitude[i], data_mybusiness.longitude[i]], popup=data_mybusiness.nome[i], tooltip=data_mybusiness.nome[i]).add_to(bdmap_)
bdmap_